In [229]:
from bs4 import BeautifulSoup as bs
import lxml
import pandas as pd
from Levenshtein import distance as lev

In [182]:
with open("UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", "r") as file:
    handlerA = file.read()

with open("UC 3/etl_generated/R22-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", "r") as file:
    handlerB = file.read()

soupA = bs(handlerA, "lxml-xml")
list_of_all_idsA = soupA.find_all("SHORT-NAME")
soupB = bs(handlerB, "lxml-xml")
list_of_all_idsB = soupB.find_all("SHORT-NAME")

list_of_all_idsA = [id.getText() for id in list_of_all_idsA]
list_of_all_idsB = [id.getText() for id in list_of_all_idsB]

dfA = pd.DataFrame({"idsA": list_of_all_idsA})
dfB = pd.DataFrame({"idsB": list_of_all_idsB})
df_merged = pd.merge(dfA, dfB, how = "outer", left_on = "idsA", right_on = "idsB")
df_merged["type"] = "unchanged"
df_merged["type"][df_merged.idsB.isna()] = "deleted"
df_merged["type"][df_merged.idsA.isna()] = "new"
df_merged.head()

,idsA,idsB,type
0,SWS_CANIF_00378,SWS_CANIF_00378,unchanged
1,SWS_CANIF_00672,SWS_CANIF_00672,unchanged
2,SWS_CANIF_00903,SWS_CANIF_00903,unchanged
3,SWS_CANIF_00023,SWS_CANIF_00023,unchanged
4,SWS_CANIF_00662,SWS_CANIF_00662,unchanged


# New Part: Change Detection

Retrieve all strings for given SHORT-NAME

In [183]:
df = df_merged.copy()
df['Text Release A']=""
df['Text Release B']=""
soup = soupA

for row in df.iterrows():
    if (soup.find('SHORT-NAME', string=row[1][0])) and (not pd.isna(row[1][0])):
        df['Text Release A'].iloc[row[0]] = [x.getText() for x in soup.find('SHORT-NAME', string=row[1][0]).parent.find_all('L-1')]    

soup = soupB
for row in df.iterrows():
    if (soup.find('SHORT-NAME', string=row[1][1])) and (not pd.isna(row[1][1])):
        df['Text Release B'].iloc[row[0]] = [x.getText() for x in soup.find('SHORT-NAME', string=row[1][1]).parent.find_all('L-1')]
df

,idsA,idsB,type,Text Release A,Text Release B
0,SWS_CANIF_00378,SWS_CANIF_00378,unchanged,[\nCanIf shall access the location of the API ...,[\nCanIf shall access the location of the API ...
1,SWS_CANIF_00672,SWS_CANIF_00672,unchanged,[The header file CanIf.h only contains extern ...,[The header file CanIf.h only contains extern ...
2,SWS_CANIF_00903,SWS_CANIF_00903,unchanged,[\nCanIf shall include the header file Mirror....,[\nCanIf shall include the header file Mirror....
3,SWS_CANIF_00023,SWS_CANIF_00023,unchanged,[\nCanIf shall avoid direct access to hardware...,[\nCanIf shall avoid direct access to hardware...
4,SWS_CANIF_00662,SWS_CANIF_00662,unchanged,[\nCanIf shall use two types of HOH to enable ...,[\nCanIf shall use two types of HOH to enable ...
...,...,...,...,...,...
508,NaN,ECUC_CanIf_00860,new,,"[\nSWS Item\n, \nECUC_CanIf_00860\n, \nContain..."
509,NaN,ECUC_CanIf_00864,new,,"[\nSWS Item\n, \nECUC_CanIf_00864\n, \nParamet..."
510,NaN,ECUC_CanIf_00861,new,,"[\nSWS Item\n, \nECUC_CanIf_00861\n, \nParamet..."
511,NaN,ECUC_CanIf_00863,new,,"[\nSWS Item\n, \nECUC_CanIf_00863\n, \nParamet..."


Example IDs:

SWS_CANIF_00378 --> no change in content
SWS_CANIF_00851 --> content changed
ECUC_CanIf_00619 --> content only slightly changed, but different structure

In [176]:
df[df['idsA']=='SWS_CANIF_00142']['Text Release A'].values[0]

['\nModule\n',
 '\nHeader File\n',
 '\nImported Type\n',
 'Can',
 'Can_GeneralTypes.h',
 '\nCan_ControllerStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwHandleType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwType\n',
 'Can_GeneralTypes.h',
 '\nCan_IdType\n',
 'Can_GeneralTypes.h',
 '\nCan_PduType\n',
 'CanTrcv',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupReasonType\n',
 'ComStack_Types',
 'ComStack_Types.h',
 '\nPduIdType\n',
 'ComStack_Types.h',
 '\nPduInfoType\n',
 'ComStack_Types.h',
 '\nPduLengthType\n',
 'EcuM',
 'EcuM.h',
 '\nEcuM_WakeupSourceType\n',
 'IdsM',
 'IdsM_Types.h',
 '\nIdsM_SecurityEventIdType\n',
 'Std',
 'Std_Types.h',
 '\nStd_ReturnType\n',
 'Std_Types.h',
 '\nStd_VersionInfoType\n']

In [177]:
df[df['idsB']=='SWS_CANIF_00142']['Text Release B'].values[0]

['\nModule\n',
 '\nHeader File\n',
 '\nImported Type\n',
 'Can',
 'Can_GeneralTypes.h',
 '\nCan_ControllerStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwHandleType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwType\n',
 'Can_GeneralTypes.h',
 '\nCan_IdType\n',
 'Can_GeneralTypes.h',
 '\nCan_PduType\n',
 'Can_GeneralTypes.h',
 '\nCan_TimeStampType (draft)',
 'CanTrcv',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupReasonType\n',
 'ComStack_Types',
 'ComStack_Types.h',
 '\nPduIdType\n',
 'ComStack_Types.h',
 '\nPduInfoType\n',
 'ComStack_Types.h',
 '\nPduLengthType\n',
 'EcuM',
 'EcuM.h',
 '\nEcuM_WakeupSourceType\n',
 'IdsM',
 'IdsM_Types.h',
 '\nIdsM_SecurityEventIdType\n',
 'Std',
 'Std_Types.h',
 '\nStd_ReturnType\n',
 'Std_Types.h',
 '\nStd_VersionInfoType\n']

In [184]:
def jaccard_similarity(a, b):
    # convert to set
    a = set(a)
    b = set(b)
    # calucate jaccard similarity
    j = float(len(a.intersection(b))) / len(a.union(b))
    return j

In [234]:
df_merged_2 = df.copy()
df_merged_2 = df_merged_2[df_merged_2['type']=='unchanged']
df_merged_2['Difference in String'] = (df_merged_2['Text Release A'] != df_merged_2['Text Release B'])
df_merged_2['Change Rate Jaccard'] = 1 - df_merged_2.apply(lambda row: jaccard_similarity(str(row[3]), str(row[4])), axis=1)
df_merged_2['Change Rate Edit Distance'] = df_merged_2.apply(lambda row: lev(str(row[3]), str(row[4]))/(max(len(str(row[3])), len(str(row[4])))), axis=1)

df_merged_2.sort_values(by='Change Rate Jaccard', axis=0, ascending=True)

,idsA,idsB,type,Text Release A,Text Release B,Difference in String,Change Rate Jaccard,Change Rate Edit Distance
0,SWS_CANIF_00378,SWS_CANIF_00378,unchanged,[\nCanIf shall access the location of the API ...,[\nCanIf shall access the location of the API ...,False,0.000000,0.000000
237,SWS_CANIF_00891,SWS_CANIF_00891,unchanged,[Configuration of <User_TriggerTransmit>(): If...,[Configuration of <User_TriggerTransmit>(): If...,False,0.000000,0.000000
236,SWS_CANIF_00890,SWS_CANIF_00890,unchanged,[Configuration of <User_TriggerTransmit>(): If...,[Configuration of <User_TriggerTransmit>(): If...,False,0.000000,0.000000
235,SWS_CANIF_00889,SWS_CANIF_00889,unchanged,[Configuration of <User_TriggerTransmit>(): Th...,[Configuration of <User_TriggerTransmit>(): Th...,False,0.000000,0.000000
234,SWS_CANIF_00888,SWS_CANIF_00888,unchanged,[Configuration of <User_TriggerTransmit>(): Th...,[Configuration of <User_TriggerTransmit>(): Th...,False,0.000000,0.000000
...,...,...,...,...,...,...,...,...
352,ECUC_CanIf_00244,ECUC_CanIf_00244,unchanged,"[Module Name, CanIfCanIf\n, Module Description...","[\nSWS Item\n, \nECUC_CanIf_00244\n, \nModule ...",True,0.210526,0.827318
417,ECUC_CanIf_00250,ECUC_CanIf_00250,unchanged,"[Container Name, CanIfDispatchCfgCanIfDispatch...","[\nSWS Item\n, \nECUC_CanIf_00250\n, \nContain...",True,0.228070,0.789838
353,ECUC_CanIf_00245,ECUC_CanIf_00245,unchanged,"[Container Name, CanIfPrivateCfgCanIfPrivateCf...","[\nSWS Item\n, \nECUC_CanIf_00245\n, \nContain...",True,0.322034,0.784504
358,ECUC_CanIf_00246,ECUC_CanIf_00246,unchanged,"[Container Name, CanIfPublicCfgCanIfPublicCfg\...","[\nSWS Item\n, \nECUC_CanIf_00246\n, \nContain...",True,0.373134,0.885378


In [198]:
df_merged_2[['Text Release A', 'Text Release B', 'Difference in String', 'Change Rate Jaccard']].head()
print(df_merged_2['Text Release A'][0])
print(df_merged_2['Text Release B'][0])
jaccard_similarity(df_merged_2['Text Release A'][0], df_merged_2['Text Release B'][0])

['\nCanIf shall access the location of the API of all used underlying CanDrv for link time configuration by a set of function pointers for each CanDrv.']
['\nCanIf shall access the location of the API of all used underlying CanDrv for link time configuration by a set of function pointers for each CanDrv.']


1.0

In [188]:
no_changed = df_merged_2['Difference in String'][df_merged_2['Difference in String']==True].count()
no_total = df_merged_2[df_merged_2['type']=='unchanged']['Difference in String'].count()

In [179]:
no_changed/no_total

0.26681614349775784

In [180]:
df_merged_change = df_merged_2[df_merged_2['Difference in String']==True] 
df_merged_change = df_merged_change[(df_merged_change['idsA'].str.contains('SWS', na=False, regex=False)) | (df_merged_change['idsB'].str.contains('SWS', na=False, regex=False))] 
df_merged_change

,idsA,idsB,type,Text Release A,Text Release B,Difference in String
398,SWS_CANIF_00142,SWS_CANIF_00142,unchanged,"[\nModule\n, \nHeader File\n, \nImported Type\...","[\nModule\n, \nHeader File\n, \nImported Type\...",True
400,SWS_CANIF_00137,SWS_CANIF_00137,unchanged,"[\nName\n, CanIf_PduModeTypeCanIf_PduModeType\...","[\nName\n, CanIf_PduModeTypeCanIf_PduModeType\...",True
440,SWS_CANIF_00294,SWS_CANIF_00294,unchanged,"[\nAPI Function\n, \nHeader File\n, \nDescript...","[\nAPI Function\n, \nHeader File\n, \nDescript...",True
444,SWS_CANIF_00532,SWS_CANIF_00532,unchanged,"[\nService Name\n, <User_ValidateWakeupEvent>U...","[\nService Name\n, <User_ValidateWakeupEvent>U...",True
445,SWS_CANIF_00014,SWS_CANIF_00014,unchanged,"[\nService Name\n, <User_ControllerBusOff>User...","[\nService Name\n, <User_ControllerBusOff>User...",True
446,SWS_CANIF_00821,SWS_CANIF_00821,unchanged,"[\nService Name\n, <User_ConfirmPnAvailability...","[\nService Name\n, <User_ConfirmPnAvailability...",True


In [169]:
df_merged_2[df_merged_2['Text Release B'].str.contains('Included parameters', na=False)]

,idsA,idsB,type,Text Release A,Text Release B,Difference in String


In [163]:
df_merged_2['Text Release A'].str.contains('Included parameters', na=False)

0      False
1      False
2      False
3      False
4      False
       ...  
461    False
462    False
463    False
464    False
465    False
Name: Text Release A, Length: 465, dtype: bool

Update type from unchanged --> changed